### extract fibro data

In [1]:
import os
import pandas as pd 
import numpy as np

In [2]:
from clean_questions import *
from disease_type import *

In [3]:
qs_path = os.path.join('..', 'funpack_cfg', 'qsidp_subjs_disease_allvisits_extended.tsv')
df_subjects = pd.read_csv(qs_path, sep='\t')

/home/fs0/syzhang/.conda/envs/bbk/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (863,865,867,869) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
df_subjects.shape

(23793, 2861)

In [5]:
## pain group 
# either reported FM using field id 20002 or general pain for 3+ months (field id 2956)
code = 2956
code_root = str(code)+'-'
cols_ls = [col for col in df_subjects.columns if col[:len(code_root)]==code_root]
print(cols_ls)

['2956-0.0', '2956-1.0', '2956-2.0', '2956-3.0']


In [6]:
gen_pain = []
for c in cols_ls:
    df_tmp = df_subjects[df_subjects[c]==1]
    print(c, df_tmp.shape)
    gen_pain.append(df_tmp)

2956-0.0 (268, 2861)
2956-1.0 (35, 2861)
2956-2.0 (302, 2861)
2956-3.0 (25, 2861)


In [7]:
df_genpain = pd.concat(gen_pain)
df_genpain.drop_duplicates(inplace=True)

In [8]:
df_genpain.shape

(518, 2861)

In [9]:
# df_disease_label = disease_label(df_genpain, visits=[2], grouping='detailed')
df_dl = disease_label(df_subjects, visits=[2], grouping='detailed')

In [10]:
df_dl.sum()

irritable bowel syndrome       1087
migraine                       1379
headaches (not migraine)        190
back problem                    112
cervical spondylosis            142
spine arthritis/spondylitis     207
prolapsed disc/slipped disc     410
disc problem                     67
disc degeneration               165
back pain                       801
ankylosing spondylitis           55
psoriatic arthropathy            53
rheumatoid arthritis            318
synovitis                         8
osteoarthritis                 2367
joint pain                      413
arthritis (nos)                1176
joint disorder                   10
fibromyalgia                    207
chronic fatigue syndrome        125
dtype: int64

In [11]:
fibro_idx = extract_disease(df_subjects, 1542, visit=[2])
idx = fibro_idx==1

In [12]:
df_fibro = df_subjects[idx.values.flatten()]

In [13]:
df_fibro.shape

(207, 2861)

In [14]:
df_fibro_and_genpain = pd.concat([df_genpain, df_fibro])

In [15]:
df_fibro_and_genpain.shape

(725, 2861)

In [16]:
df_fibro_and_genpain_dropd = df_fibro_and_genpain[~df_fibro_and_genpain.index.duplicated()]

In [17]:
df_fibro_and_genpain_dropd.shape

(619, 2861)

In [19]:
# def disease_label(df, visits=[2], grouping='simplified'):
#     """create disease label df"""
#     from disease_type import extract_disease
#     # different grouping
#     if grouping == 'simplified':
#         df_disease_group = pd.read_csv('./bbk_codes/disease_code_simplified.csv')
#     elif grouping == 'detailed':
#         df_disease_group = pd.read_csv('./bbk_codes/disease_code.csv')
#     else:
#         df_disease_group = pd.read_csv('./bbk_codes/disease_code_grouped.csv')
#     # drop duplicates to avoid merging issues
#     if df.index.name == 'eid':
#         df_dropd = df[~df.index.duplicated()]
#     elif 'eid' in df:
#         df_dropd = df[~df['eid'].duplicated()]
#     else:
#         raise ValueError('eid not present.')
#     # iterate diseases
#     df_disease_label = []
#     for i, r in df_disease_group.iterrows():
#         df_tmp = extract_disease(df_dropd, r['code'], visit=visits)
#         df_tmp.replace(np.nan, 0.0, inplace=True)
#         # rename column to disease
#         df_tmp.rename(columns={df_tmp.columns[0]: r['disease']}, inplace=True)
#         # merge
#         df_disease_label.append(df_tmp)
#     return pd.concat(df_disease_label,axis=1)

In [20]:
fibro_and_genpain_labels = disease_label(df_fibro_and_genpain_dropd, visits=[2], grouping='detailed')
# fibro_and_genpain_labels = disease_label(df_subjects, visits=[2], grouping='grouped')

In [21]:
fibro_and_genpain_labels.sum()

irritable bowel syndrome        53
migraine                        44
headaches (not migraine)         9
back problem                     3
cervical spondylosis            12
spine arthritis/spondylitis     21
prolapsed disc/slipped disc     12
disc problem                     1
disc degeneration               13
back pain                       30
ankylosing spondylitis           2
psoriatic arthropathy            3
rheumatoid arthritis            32
synovitis                        0
osteoarthritis                 118
joint pain                      17
arthritis (nos)                 41
joint disorder                   1
fibromyalgia                   207
chronic fatigue syndrome        43
dtype: int64

In [34]:
np.sum(fibro_and_genpain_labels.sum(axis=1)>1)

163

In [24]:
# extract idps and qs
questionnaire = ['cwp', 'demographic', 'mental', 'lifestyle', 'cognitive']
idp = ['subcorticalvol', 'fast', 't1vols']
question_visits = [2]

In [25]:
qs = load_qscode(questionnaire=questionnaire, idp=idp)

In [26]:
df_fibro_and_genpain_idps = extract_qs(df_fibro_and_genpain_dropd, df_questionnaire=qs, visits=question_visits)

In [27]:
df_fibro_and_genpain_idps.shape

(619, 333)

In [28]:
# only 1 disease
labels = fibro_and_genpain_labels.idxmax(axis=1)
labels.name = 'label'

In [29]:
labels

eid
1013760                 osteoarthritis
1070136                 osteoarthritis
1108699    spine arthritis/spondylitis
1108787       irritable bowel syndrome
1136360    spine arthritis/spondylitis
                      ...             
5802589                   fibromyalgia
5808560                       migraine
5878634                   fibromyalgia
5882291                       migraine
5945754       irritable bowel syndrome
Name: label, Length: 619, dtype: object

In [30]:
# df_out = df_fibro_and_genpain_idps.merge(labels.reset_index(), on='eid')
df_out = df_fibro_and_genpain_idps.merge(fibro_and_genpain_labels.reset_index(), on='eid')

In [31]:
df_out

,26418-0.0,2020-2.0,2010-2.0,25869-2.0,20122-0.0,25822-2.0,1990-2.0,25789-2.0,20124-0.0,20531-0.0,...,ankylosing spondylitis,psoriatic arthropathy,rheumatoid arthritis,synovitis,osteoarthritis,joint pain,arthritis (nos),joint disorder,fibromyalgia,chronic fatigue syndrome
0,NaN,0.0,0.0,1188.17,NaN,3367.99,1.0,8543.64,NaN,0.0,...,0,0,0,0,1,0,0,0,0,0
1,NaN,0.0,0.0,1245.25,NaN,2957.61,0.0,7841.86,NaN,NaN,...,0,0,0,0,1,0,1,0,0,0
2,NaN,0.0,0.0,1771.70,NaN,4208.46,-1.0,7941.84,NaN,1.0,...,0,0,0,0,1,0,0,0,1,0
3,NaN,1.0,1.0,1813.30,NaN,4570.22,0.0,10175.10,NaN,0.0,...,0,0,0,0,0,0,0,0,0,0
4,NaN,1.0,1.0,1220.78,NaN,4427.66,1.0,9064.04,NaN,1.0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
614,NaN,0.0,0.0,1506.37,NaN,4320.07,0.0,8341.86,NaN,0.0,...,0,0,0,0,0,0,0,0,1,0
615,NaN,0.0,-1.0,1398.44,NaN,3338.43,0.0,8614.51,NaN,1.0,...,0,0,0,0,0,0,0,0,1,0
616,NaN,0.0,0.0,1157.57,NaN,4258.69,1.0,8376.58,NaN,1.0,...,0,0,0,0,0,0,0,0,1,0
617,NaN,0.0,-1.0,1132.01,NaN,3599.42,0.0,7789.47,NaN,0.0,...,0,0,0,0,1,1,0,0,1,0


In [32]:
df_out.shape

(619, 353)

In [33]:
# df_out.to_csv('../fibro/pain_subjects.csv', index=None)
df_out.to_csv('../fibro/pain_subjects_full_labels.csv', index=None)

#### controls

In [87]:
qs_path = os.path.join('..', 'funpack_cfg', 'qsidp_subjs_control_allvisits_extended.tsv')
df_controls = pd.read_csv(qs_path, sep='\t')

/home/fs0/syzhang/.conda/envs/bbk/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (863,869) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [88]:
df_controls.shape

(26490, 2861)

In [89]:
control_disease = disease_label(df_controls, visits=[2], grouping='detailed')

In [90]:
control_disease.sum()

irritable bowel syndrome        527
migraine                        609
headaches (not migraine)         80
back problem                     59
cervical spondylosis             54
spine arthritis/spondylitis      87
prolapsed disc/slipped disc     265
disc problem                     41
disc degeneration                75
back pain                       501
ankylosing spondylitis           29
psoriatic arthropathy            27
rheumatoid arthritis            164
synovitis                         6
osteoarthritis                 1106
joint pain                      273
arthritis (nos)                 718
joint disorder                    9
fibromyalgia                     40
chronic fatigue syndrome         56
dtype: int64

In [91]:
nd_idx = control_disease.sum(axis=1)==0

In [92]:
dfc_nd = df_controls[nd_idx.values]

In [93]:
dfc_nd.shape

(22705, 2861)

In [94]:
## pain group 
# either reported FM using field id 20002 or general pain for 3+ months (field id 2956)
code = 2956
code_root = str(code)+'-'
cols_ls = [col for col in dfc_nd.columns if col[:len(code_root)]==code_root]
print(cols_ls)

['2956-0.0', '2956-1.0', '2956-2.0', '2956-3.0']


In [95]:
nd_np = []
for c in cols_ls:
    df_tmp = dfc_nd[dfc_nd[c]!=1]
    print(c, df_tmp.shape)
    nd_np.append(df_tmp)

2956-0.0 (22680, 2861)
2956-1.0 (22701, 2861)
2956-2.0 (22686, 2861)
2956-3.0 (22703, 2861)


In [96]:
## pain group 
# either reported FM using field id 20002 or general pain for 3+ months (field id 2956)
code = 6159
code_root = str(code)+'-2'
cols_ls = [col for col in dfc_nd.columns if col[:len(code_root)]==code_root]
print(cols_ls)

['6159-2.0', '6159-2.1', '6159-2.2', '6159-2.3', '6159-2.4', '6159-2.5', '6159-2.6']


In [97]:
nd_np = []
for c in cols_ls:
    df_tmp = dfc_nd[dfc_nd[c]==-7]
    print(c, df_tmp.shape)
    nd_np.append(df_tmp)

6159-2.0 (17173, 2861)
6159-2.1 (0, 2861)
6159-2.2 (0, 2861)
6159-2.3 (0, 2861)
6159-2.4 (0, 2861)
6159-2.5 (0, 2861)
6159-2.6 (0, 2861)


In [98]:
df_npnd = pd.concat(nd_np)
df_npnd.drop_duplicates(inplace=True)

In [99]:
df_npnd.shape

(17173, 2861)

In [100]:
# find matched controls
from compare_control import extract_control
df_matched = extract_control(df_control=df_npnd, df_disease=df_out, save_csv=False)

In [101]:
len(df_matched)

618

In [102]:
dfc_matched = df_npnd[df_npnd['eid'].isin(df_matched)]

In [103]:
dfc_matched.shape

(618, 2861)

In [104]:
dfc_out = extract_qs(dfc_matched, df_questionnaire=qs, visits=question_visits)

In [105]:
dfc_out['label'] = 'control'

In [106]:
dfc_out.shape

(618, 334)

In [111]:
dfc_out.to_csv('../fibro/control_subjects.csv', index=None)

In [107]:
df_out['31-0.0'].value_counts()

0.0    447
1.0    172
Name: 31-0.0, dtype: int64

In [108]:
dfc_out['31-0.0'].value_counts()

0.0    446
1.0    172
Name: 31-0.0, dtype: int64

In [109]:
df_out['21003-2.0'].mean()

62.79806138933764

In [110]:
dfc_out['21003-2.0'].mean()

62.82686084142395

#### check 

In [15]:
dfc = pd.read_csv('../fibro/control_subjects.csv')
dfp = pd.read_csv('../fibro/pain_subjects_full_labels.csv')

In [16]:
dfc.shape

(618, 334)

In [17]:
dfp.shape

(619, 353)

In [18]:
dfc['21000-2.0'].value_counts()

1001.0    156
1002.0      9
1003.0      5
3004.0      1
2003.0      1
2004.0      1
6.0         1
Name: 21000-2.0, dtype: int64

In [19]:
dfp['21000-2.0'].value_counts()

1001.0    151
1003.0      6
3001.0      6
1002.0      3
4001.0      2
4002.0      1
5.0         1
Name: 21000-2.0, dtype: int64

In [20]:
qs_path = os.path.join('..', 'funpack_cfg', 'qsidp_subjs_disease_allvisits_extended.tsv')
df_subjects = pd.read_csv(qs_path, sep='\t')

/home/fs0/syzhang/.conda/envs/bbk/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (863,865,867,869) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [21]:
df_subjects.columns

Index(['eid', '26414-0.0', '26431-0.0', '26421-0.0', '26412-0.0', '26429-0.0',
       '26413-0.0', '26430-0.0', '26420-0.0', '26411-0.0',
       ...
       '21023-0.0', '21027-0.0', '21035-0.0', '21048-0.0', '21052-0.0',
       '21051-0.0', '21049-0.0', '21057-0.0', '21025-0.0', '21036-0.0'],
      dtype='object', length=2861)

In [23]:
from subject_select import check_field, check_count

In [28]:
check_field(df_subjects, 21000, visit=0)

['21000-0.0']

In [33]:
df_subjects['21000-0.0'].isna().sum()

9

In [37]:
check_field(df_subjects, 10004, visit=0)

[]

In [42]:
sjs = pd.concat([dfp['eid'], dfc['eid']])

In [53]:
sjs.to_csv('../fibro/subject_eids.txt', index=None, header=None)

In [54]:
f = [20003,137,6154,10004,21000]

In [55]:
ff = pd.DataFrame({'field':f}, dtype=int)

In [56]:
ff.to_csv('../fibro/extra_fields.txt', index=None, header=None)

In [57]:
ff

,field
0,20003
1,137
2,6154
3,10004
4,21000


#### check how many have imaging

In [3]:
dff_check = pd.read_csv('../fibro/pain_subjects.csv')

In [5]:
dff_check.shape

(619, 334)

In [6]:
dff_check2 = pd.read_csv('../fibro/control_subjects.csv')

In [7]:
dff_check2.shape

(618, 334)